# Deploy a dockerfile into a artifact registry and try to deploy into a compute engine

## I) INTRODUCTION

These codes could be run in the Google SDK console, as well as run on a notebook.
For a data scientist who is not specialized in devops practices, using a notebook is more intuitive to use

#### Previous steps
- Stop in the root folder of the application (this notebook is created in the root)
- Have scripts created to contain and upload to cloud run

#### Important information
**To run a console command in a Jupyter notebook and the python variables stored in the notebook can also be passed to the command, you must use the peso sign ($) and not use the assignment command (=)**

## II) INITIALIZE READ .ENV WITH ENV VARIABLES

In [3]:
import os
from dotenv import load_dotenv, find_dotenv # package used in jupyter notebook to read the variables in file .env

""" get env variable from .env """
load_dotenv(find_dotenv())

""" Read env variables and save it as python variable """
PROJECT_ID = os.environ.get("PROJECT_GCP", "")

## III) DEFINE PARAMETERS TO DEPLOY APP INTO A CLOUD RUN 

### Step 0: Connect to GCP project

In [4]:
! gcloud config set project $PROJECT_ID

Updated property [core/project].


### Step 1: Define parameters

In [9]:
# PARARAMETERS

# general gcp
REGION = 'us-east1'

# name of the repo in artifact registry where will be saved docker images
NAME_REPO = 'repo-for-test-analytics-environment'
FORMAT_REPO = 'docker'
DESCRIPTION_REPO = "repo for test analytics environment"

# name of the docker image saved in docker repo in artifact registry
NAME_IMAGE = 'test-analytics-environment'

# name of cloud run where the app web will be located
NAME_CLOUD_RUN = 'test-analytics-environment'

# IV) Upload a docker image with the codes of the app in Artifact Registry

- Artifact registry is the replacement for container registry and recommended by google. The only difference is that the image is saved in this new service and you need to run another command

- **Additionally, every time a new image is uploaded to the artifact registry (rerun the corresponding gcloud command), it receives the latest tag and is the one used to create/update the created cloud run**

- Uploading the image to the artifact Registry requires more steps than uploading it to the container registry

- Cloud build integration documentation with Artifact Registry: https://cloud.google.com/artifact-registry/docs/configure-cloud-build?hl=es-419

### Step 1. Create repository in artifact registry (if it does not exist)
- Unlike container registry which was automatic, in artifact registry you have to create it. **If the repo already exists the gcloud command return an error but doesn't stop de execution of the notebook**

- A repo is created which can have multiple images and each one have different versions

- Documentation: create repo in artifact registry: https://cloud.google.com/artifact-registry/docs/repositories/create-repos#gcloud

In [6]:
# create repo artifact registry
! gcloud artifacts repositories create $NAME_REPO \
--repository-format $FORMAT_REPO \
--location $REGION \
--description "$DESCRIPTION_REPO" \
--async

Create request issued for: [repo-for-test-analytics-environment]
Check operation [projects/cmpc-innovation-cd4ml-test/locations/us-east1/operations/74543aee-296c-43ad-a5fd-5e11c626d553] for status.


### Step 2: Set up a Docker build

It is necessary to create a **yaml** with the configuration to build the docker image in Artifact Registry.

It has the following form

<code>
steps:
- name: 'gcr.io/cloud-builders/docker'
   args: [ 'build', '-t', '${_LOCATION}-docker.pkg.dev/$PROJECT_ID/${_REPOSITORY}/${_IMAGE}', '.' ]
images:
- '${_LOCATION}-docker.pkg.dev/$PROJECT_ID/${_REPOSITORY}/${_IMAGE}'
<code>
    

**This is a GENERIC FILE that can be recycled because it is parameterized to work with any docker repo in the artifact registry**

--> Running the following line of code creates a yaml file with the desired configuration.

Documentation: https://stackabuse.com/reading-and-writing-yaml-to-a-file-in-python/

In [7]:
import yaml

# create a python diccionary with the content of the yaml cloudbuild generic

dict_python_yaml_cloudbuild = {'steps': [{'name': 'gcr.io/cloud-builders/docker',
   'args': ['build', '-t', '${_LOCATION}-docker.pkg.dev/$PROJECT_ID/${_REPOSITORY}/${_IMAGE}', '.']}],
 'images': ['${_LOCATION}-docker.pkg.dev/$PROJECT_ID/${_REPOSITORY}/${_IMAGE}']}

# save dicctionary in yaml format
with open(r'cloudbuild.yaml', 'w') as file:
    documents = yaml.dump(dict_python_yaml_cloudbuild, file)

### Step 3: Create Dockerfile
Manually created the dockerfile

### Step 4: Containerize (docker image) web app codes using cloud build and upload them to artifact registry
- In this step, a docker image is created with the necessary codes for the web app and then this image is uploaded to Artifact Registry (using as a base the "cloudbuild.yaml" file that calls the "Dockerfile", created in the steps previous)

In [8]:
##### VERY IMPORTANT NOTATION

# NOTE: The variable names in the gcloud command correspond to the variables defined in the configuration file
#yaml

# NOTE2: to pass the name of the variables (as always) you must use the dollar sign "$" but you must
# to be enclosed in quotes (so that it is understood that it is the variable to be replaced in the configuration yaml)

# NOTE3: it must be double quotes and without spaces to avoid problems

! gcloud builds submit \
    --config=cloudbuild.yaml \
    --substitutions=_LOCATION="$REGION",_REPOSITORY="$NAME_REPO",_IMAGE="$NAME_IMAGE" .

----------------------------- REMOTE BUILD OUTPUT ------------------------------
starting build "d7cdcfe0-47bf-4eef-8891-f1024c6131a1"

FETCHSOURCE
Fetching storage object: gs://cmpc-innovation-cd4ml-test_cloudbuild/source/1710047497.923339-3541bef0b79e4f8bbcca10cf2184a070.tgz#1710047500183379
Copying gs://cmpc-innovation-cd4ml-test_cloudbuild/source/1710047497.923339-3541bef0b79e4f8bbcca10cf2184a070.tgz#1710047500183379...
/ [0 files][    0.0 B/ 22.6 KiB]                                                
/ [1 files][ 22.6 KiB/ 22.6 KiB]                                                

Operation completed over 1 objects/22.6 KiB.
BUILD
Already have image (with digest): gcr.io/cloud-builders/docker
Sending build context to Docker daemon  35.84kB


Step 1/7 : FROM python:3.10.13-slim-bullseye
3.10.13-slim-bullseye: Pulling from library/python
5d0aeceef7ee: Pulling fs layer
de0ce72dbd52: Pulling fs layer
e808301c35ad: Pulling fs layer
104978216976: Pulling fs layer
0b15937fe6c9: Pulling fs 

Creating temporary tarball archive of 25 file(s) totalling 87.9 KiB before compression.
Some files were not included in the source upload.

Check the gcloud log [C:\Users\JORTEGAL\AppData\Roaming\gcloud\logs\2024.03.10\02.11.37.641209.log] to see which files and the contents of the
default gcloudignore file used (see `$ gcloud topic gcloudignore` to learn
more).

Uploading tarball of [.] to [gs://cmpc-innovation-cd4ml-test_cloudbuild/source/1710047497.923339-3541bef0b79e4f8bbcca10cf2184a070.tgz]
Created [https://cloudbuild.googleapis.com/v1/projects/cmpc-innovation-cd4ml-test/locations/global/builds/d7cdcfe0-47bf-4eef-8891-f1024c6131a1].
Logs are available at [ https://console.cloud.google.com/cloud-build/builds/d7cdcfe0-47bf-4eef-8891-f1024c6131a1?project=724348686027 ].


### Step 5: Deploy the artifact registry container image to cloud runThe main idea is the analytics environment are located NOT in a serverless service where the codes and artifact developed will loss when the serives shutdown. 

But only for the exampleCCESS**

In [9]:
#### como setear variables de ambiente en cloud run
#--set-env-vars=PROJECT_GCP=$PROJECT_ID \

In [10]:
! gcloud run deploy $NAME_CLOUD_RUN \
    --image $REGION-docker.pkg.dev/$PROJECT_ID/$NAME_REPO/$NAME_IMAGE \
    --region $REGION \
    --set-env-vars=PROJECT_GCP=$PROJECT_ID \
    --allow-unauthenticated

Deploying container to Cloud Run service [test-analytics-environment] in project [cmpc-innovation-cd4ml-test] region [us-east1]
Deploying new service...
Setting IAM Policy.......................done
Creating Revision...............done
Routing traffic.............................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................

### Step 5: Deploy the artifact registry into a Compute Engine GCP